In [1]:
import numpy as np
from keras.datasets import cifar10
from keras import models
from keras import layers
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
# load pre-trained conv net base
img_rows, img_cols, channels = 96, 96, 3

from keras.applications.vgg16 import VGG16, preprocess_input
inception_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, channels))

58892288/58889256 [==============================] - 4s 0us/step


In [4]:
# reshape images dimension to 96, 96, 3
import scipy.misc
x_train_big = np.array([scipy.misc.imresize(img, (img_rows, img_cols, channels))
                            for img in x_train]).astype('float32')
vgg16_input_train = preprocess_input(x_train_big)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  after removing the cwd from sys.path.


In [5]:
# one-hot encoding
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [6]:
## Model building
import keras.optimizers
model = models.Sequential()
model.add(inception_model)
model.add(layers.Conv2D(filters=256, kernel_size=2))
model.add(layers.Dropout(0.4))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer=keras.optimizers.RMSprop(lr=2e-5),
               loss = 'categorical_crossentropy',
               metrics=['accuracy'])

In [8]:
## Training
history = model.fit(vgg16_input_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 278s 7ms/step - loss: 1.1236 - acc: 0.6354 - val_loss: 0.4652 - val_acc: 0.8488
Epoch 2/10
40000/40000 [==============================] - 274s 7ms/step - loss: 0.4224 - acc: 0.8647 - val_loss: 0.3330 - val_acc: 0.8928
Epoch 3/10
40000/40000 [==============================] - 273s 7ms/step - loss: 0.2578 - acc: 0.9168 - val_loss: 0.3256 - val_acc: 0.8977
Epoch 4/10
40000/40000 [==============================] - 273s 7ms/step - loss: 0.1660 - acc: 0.9461 - val_loss: 0.3027 - val_acc: 0.9158
Epoch 5/10
40000/40000 [==============================] - 273s 7ms/step - loss: 0.1168 - acc: 0.9617 - val_loss: 0.3387 - val_acc: 0.9118
Epoch 6/10
  960/40000 [..............................] - ETA: 4:06 - loss: 0.0572 - acc: 0.9771

KeyboardInterrupt: 

In [22]:
x_test_big = np.array([scipy.misc.imresize(img, (img_rows, img_cols, channels))
                            for img in x_test]).astype('float32')
x_test_big = preprocess_input(x_test_big)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [23]:
loss, accuracy = model.evaluate(x_test_big, y_test)
accuracy * 100

10000/10000 [==============================] - 22s 2ms/step


91.45